# Aggregating and Combining `pandas` DataFrames

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging, joining, and concatinating

## Set Up

Surprise, surprise... we're still working with the Austin Animal Center Data! Let's start with Outcomes

In [1]:
# Imports

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [16]:
outcomes = pd.read_csv('data/Austin_Animal_Center_Outcomes_022822.csv',
                       parse_dates=['DateTime', 'Date of Birth'])

In [17]:
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby


In [18]:
outcomes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137097 entries, 0 to 137096
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Animal ID         137097 non-null  object        
 1   Name              96095 non-null   object        
 2   DateTime          137097 non-null  datetime64[ns]
 3   MonthYear         137097 non-null  object        
 4   Date of Birth     137097 non-null  datetime64[ns]
 5   Outcome Type      137073 non-null  object        
 6   Outcome Subtype   62653 non-null   object        
 7   Animal Type       137097 non-null  object        
 8   Sex upon Outcome  137095 non-null  object        
 9   Age upon Outcome  137092 non-null  object        
 10  Breed             137097 non-null  object        
 11  Color             137097 non-null  object        
dtypes: datetime64[ns](2), object(10)
memory usage: 12.6+ MB


In [19]:
# Let's create our Age in Days column
outcomes['Calculated Age in Days'] = pd.to_datetime(outcomes['DateTime'].dt.date) - outcomes['Date of Birth']

In [20]:
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736 days
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371 days
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366 days
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128 days
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6 days


In [15]:
# Grab just the integer here...
outcomes['Calculated Age in Days'] = outcomes['Calucated Age in Days'].dt.days

KeyError: 'Calucated Age in Days'

In [8]:
# Sanity check
outcomes.head()

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736 days
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371 days
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366 days
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128 days
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6 days


## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [12]:
# Just using groupby outputs some weird GroupBy object... not helpful
outcomes.groupby('Animal Type')

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is `groups`.

In [21]:
outcomes['Animal Type'].value_counts()

Dog          77091
Cat          52092
Other         7253
Bird           636
Livestock       25
Name: Animal Type, dtype: int64

In [23]:
# This returns each group indexed by the group name, e.g. 'Bird',
# along with the row indices of each value

outcomes.groupby('Animal Type').value_count()

AttributeError: 'DataFrameGroupBy' object has no attribute 'value_count'

In [24]:
# Multi-Index - Animal Types and the Outcome
animal_outcome = outcomes.groupby(['Animal Type', 'Outcome Type'])

In [25]:
# .groups outputs a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

dict_keys([('Cat', 'Rto-Adopt'), ('Dog', 'Adoption'), ('Other', 'Euthanasia'), ('Cat', 'Transfer'), ('Cat', 'Adoption'), ('Cat', 'Return to Owner'), ('Dog', 'Return to Owner'), ('Dog', 'Transfer'), ('Cat', 'Euthanasia'), ('Other', 'Adoption'), ('Dog', 'Rto-Adopt'), ('Cat', 'Died'), ('Dog', 'Euthanasia'), ('Other', 'Transfer'), ('Bird', 'Adoption'), ('Other', 'Disposal'), ('Other', 'Died'), ('Dog', 'Died'), ('Cat', 'Disposal'), ('Other', 'Return to Owner'), ('Bird', 'Euthanasia'), ('Bird', 'Transfer'), ('Livestock', 'Return to Owner'), ('Dog', 'Missing'), ('Other', 'Relocate'), ('Dog', nan), ('Livestock', 'Adoption'), ('Bird', 'Return to Owner'), ('Dog', 'Disposal'), ('Cat', 'Missing'), ('Bird', 'Disposal'), ('Bird', 'Died'), ('Other', 'Missing'), ('Other', 'Rto-Adopt'), ('Bird', 'Relocate'), ('Bird', 'Missing'), ('Other', nan), ('Livestock', 'Transfer'), ('Cat', 'Relocate'), ('Cat', nan), ('Livestock', 'Died'), ('Livestock', 'Euthanasia')])

In [17]:
# We can then get a specific group, such as cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color,Calculated Age in Days
7,A689724,*Donatello,2014-10-18 18:52:00,Oct 2014,2014-08-01,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,Black,78 days
8,A680969,*Zeus,2014-08-05 16:59:00,Aug 2014,2014-06-03,Adoption,NaN,Cat,Neutered Male,2 months,Domestic Shorthair Mix,White/Orange Tabby,63 days
20,A730621,*Liza,2016-09-10 18:59:00,Sep 2016,2016-05-18,Adoption,NaN,Cat,Spayed Female,3 months,Domestic Shorthair Mix,Calico,115 days
26,A801106,NaN,2019-08-16 14:05:00,Aug 2019,2019-05-06,Adoption,NaN,Cat,Neutered Male,3 months,Domestic Shorthair,Orange Tabby,102 days
54,A792258,Vesper,2019-04-10 20:53:00,Apr 2019,2016-09-08,Adoption,NaN,Cat,Spayed Female,2 years,Domestic Shorthair Mix,Tortie,944 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...
137072,A846689,Coco Chanel,2022-02-26 17:23:00,Feb 2022,2021-08-19,Adoption,NaN,Cat,Spayed Female,6 months,Domestic Shorthair,Blue Tabby,191 days
137073,A845330,Mitzi,2022-02-26 18:09:00,Feb 2022,2021-01-28,Adoption,NaN,Cat,Spayed Female,1 year,Domestic Shorthair,Torbie/White,394 days
137088,A851184,*Papaya,2022-02-28 11:38:00,Feb 2022,2021-02-08,Adoption,NaN,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Orange Tabby/White,385 days
137090,A847804,*Mahalia,2022-02-28 11:42:00,Feb 2022,2011-12-08,Adoption,NaN,Cat,Spayed Female,10 years,Domestic Shorthair Mix,Brown Tabby/White,3735 days


## Aggregating

As we will see below, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [20]:
outcomes.groupby('Animal Type').mean( )

DataError: No numeric types to aggregate

## Exercise

Use `.groupby()` to find the most recent birth date of each (main) animal type.


In [26]:
outcomes.groupby('Animal Type')['Date of Birth'].max()

Animal Type
Bird        2022-01-06
Cat         2022-02-18
Dog         2022-02-14
Livestock   2020-05-28
Other       2022-02-11
Name: Date of Birth, dtype: datetime64[ns]

<details>
    <summary>Answer</summary>

```python
outcomes.groupby('Animal Type')['Date of Birth'].max()
```
</details>

In [35]:
outcomes.value_counts()

Animal ID  Name     DateTime             MonthYear  Date of Birth  Outcome Type  Outcome Subtype  Animal Type  Sex upon Outcome  Age upon Outcome  Breed                            Color           Calculated Age in Days
A764464    Pepper   2018-09-18 14:28:00  Sep 2018   2017-10-15     Transfer      Partner          Dog          Spayed Female     11 months         Soft Coated Wheaten Terrier Mix  Black/Tricolor  338 days                  2
A695798    Jim      2015-01-23 12:34:00  Jan 2015   2014-02-23     Euthanasia    Suffering        Cat          Neutered Male     10 months         Domestic Shorthair Mix           Orange          334 days                  2
A773428    *Atreyu  2018-06-07 09:31:00  Jun 2018   2018-05-04     Died          At Vet           Cat          Intact Male       4 weeks           Domestic Shorthair Mix           Orange Tabby    34 days                   2
A797007    Ummi     2019-06-09 16:33:00  Jun 2019   2009-12-08     Died          In Kennel        Dog        

# Pivoting a DataFrame

## `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

In [34]:
outcomes.groupby(by=['Outcome Type', 'Sex upon Outcome']).agg('mean')

DataError: No numeric types to aggregate

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

In [32]:
# Check it out! #Another way to get aggretated table in frame

outcomes.pivot_table(index='Outcome Type', columns = 'Sex upon Outcome', aggfunc = 'mean')


DataError: No numeric types to aggregate

# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

## `.join()`

In [35]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

toy1

,age,HP
0,63,142
1,33,47


In [36]:
toy2

,age,MP
0,63,100
1,33,200


In [39]:
# We can't just join these as they are, since we haven't specified our suffixes

toy1.join(toy2, lsuffix='1', rsuffix='2')


,age1,HP,age2,MP
0,63,142,63,100
1,33,47,33,200


In [42]:
toy1.set_index('age').join(toy2.set_index('age'))
#Joins the files on the commonalities

,HP,MP
age,,
63,142,100
33,47,200


If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

## `.merge()`

Or we could use `.merge()`:

In [43]:
toy1.merge(toy2)
#merge might be a better way to sort info

,age,HP,MP
0,63,142,100
1,33,47,200


In [44]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [45]:
states = pd.read_csv('data/states.csv', index_col=0)
states

,state,nickname,capital
0,WA,evergreen,Olympia
1,TX,alamo,Austin
2,DC,district,Washington
3,OH,buckeye,Columbus
4,OR,beaver,Salem


## The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [48]:
ds_chars.merge(states, left_on='home_state', right_on='state', how='inner')

#inner what we share
#outter everything we have
#left takes left table intersections
#right takes right table intersections

,name,HP,home_state,state,nickname,capital
0,greg,200,WA,WA,evergreen,Olympia
1,miles,200,WA,WA,evergreen,Olympia
2,alan,170,TX,TX,alamo,Austin
3,rachel,200,TX,TX,alamo,Austin
4,alison,300,DC,DC,district,Washington


In [50]:
ds_chars.join(states, on='state')

KeyError: 'state'

## `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [51]:
prefs = pd.read_csv('data/preferences.csv', index_col=0)
prefs

,cuisine,genre
0,Greek,horror
1,Indian,scifi
2,American,fantasy
3,Thai,tech
4,Indian,documentary


In [52]:
ds_full = pd.concat([ds_chars, prefs])
ds_full

,name,HP,home_state,cuisine,genre
0,greg,200.0,WA,NaN,NaN
1,miles,200.0,WA,NaN,NaN
2,alan,170.0,TX,NaN,NaN
3,alison,300.0,DC,NaN,NaN
4,rachel,200.0,TX,NaN,NaN
0,NaN,NaN,NaN,Greek,horror
1,NaN,NaN,NaN,Indian,scifi
2,NaN,NaN,NaN,American,fantasy
3,NaN,NaN,NaN,Thai,tech
4,NaN,NaN,NaN,Indian,documentary


`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [53]:
ds_full = pd.concat([ds_chars, prefs], axis = 1)
ds_full

,name,HP,home_state,cuisine,genre
0,greg,200,WA,Greek,horror
1,miles,200,WA,Indian,scifi
2,alan,170,TX,American,fantasy
3,alison,300,DC,Thai,tech
4,rachel,200,TX,Indian,documentary


## Back to the Center

We have Intakes data and we have Outcomes data... time to merge!

In [ ]:
# Peek at the outcomes data we already had in here
outcomes.head()

In [ ]:
# Read in the intakes data
intakes = pd.read_csv("data/Austin_Animal_Center_Intakes_022822.csv",
                      parse_dates=['DateTime'])
# Check it out
intakes.head()

In [56]:
# Let's try merging on Animal ID
combined = outcomes.merge(intakes, on='Animal ID', how='inner'), 
suffixes = ['_outcome', '_intake']

NameError: name 'intakes' is not defined

In [ ]:
# What was the result?
combined.head()

Let's discuss/explore: did that work the way we expected?

- 

<details>
    <summary>Observation Notes</summary>

- We went from about 136k rows in each of the dataframes to 176k! Even using an inner join! Something seems off. 
    
    
</details>

In [ ]:
# We might want to try something different
# Can we clean something to make a better merge?
combined.loc[combined.duplicated(subset='Animal ID')]

In [58]:
# Try again
clean_intakes = intakes.drop_duplicates(subset=['Animal ID'])
clean_outcomes = outcomes.drop_duplicates(subset=['Animal ID'])

NameError: name 'intakes' is not defined

In [59]:
clean_combined_df = clean_intakes.merge(clean_outcomes, on='Animal ID',
                                        how='inner',
                                        suffixes=['_intake', '_outcome'])

NameError: name 'clean_intakes' is not defined

In [60]:
clean_combined_df.head()

NameError: name 'clean_combined_df' is not defined

In [ ]:
clean_combined_df.shape

# Level Up: Quick Column Name Clean Up Code

Throwing a quick use of a lambda function your way:

In [63]:
outcomes_renamed = outcomes.rename(columns = lambda x: x.replace(" ", "_").lower())
outcomes_renamed.head()

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,outcome_subtype,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,calculated_age_in_days
0,A794011,Chunk,2019-05-08 18:20:00,May 2019,2017-05-02,Rto-Adopt,NaN,Cat,Neutered Male,2 years,Domestic Shorthair Mix,Brown Tabby/White,736 days
1,A776359,Gizmo,2018-07-18 16:02:00,Jul 2018,2017-07-12,Adoption,NaN,Dog,Neutered Male,1 year,Chihuahua Shorthair Mix,White/Brown,371 days
2,A821648,NaN,2020-08-16 11:38:00,Aug 2020,2019-08-16,Euthanasia,NaN,Other,Unknown,1 year,Raccoon,Gray,366 days
3,A720371,Moose,2016-02-13 17:59:00,Feb 2016,2015-10-08,Adoption,NaN,Dog,Neutered Male,4 months,Anatol Shepherd/Labrador Retriever,Buff,128 days
4,A674754,NaN,2014-03-18 11:47:00,Mar 2014,2014-03-12,Transfer,Partner,Cat,Intact Male,6 days,Domestic Shorthair Mix,Orange Tabby,6 days


# Level Up: `pandas.set_option()`

We can adjust how `pandas` works by setting options in advance.

For complete documentation, see [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html).

## Block Scientific Notation

For example, suppose we want to prevent numbers from being displayed in scientific notation.

In [65]:
df = pd.DataFrame([[1e9, 2e9], [3e9, 4e9]])
df

,0,1
0,1.000000e+09,2.000000e+09
1,3.000000e+09,4.000000e+09


Then we can use:

In [66]:
pd.set_option('display.float_format', '{:.2f}'.format)

df

,0,1
0,1000000000.00,2000000000.00
1,3000000000.00,4000000000.00


## See More Rows

Or suppose we want `pandas` to show more rows.

In [67]:
df2 = pd.DataFrame(np.array(range(100)))
df2

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9


In that case we can use:

In [68]:
pd.set_option('display.max_rows', 100)

df2

,0
0,0
1,1
2,2
3,3
4,4
5,5
6,6
7,7
8,8
9,9
